# Case based on Abadie, Diamond y Hainmueller (2007)
# SYNTHETIC CONTROL METHODS FOR COMPARATIVE CASE STUDIES: ESTIMATING THE EFFECT OF CALIFORNIA'S TOBACCO CONTROL PROGRAM.

https://www.nber.org/system/files/working_papers/t0335/t0335.pdf

### Problem Overview

In 1988, California passed a famous Tobacco Tax and Health Protection Act, which became known as Proposition 99. Its primary effect is to impose a 25-cent per pack state excise tax on the sale of tobacco cigarettes within California, with approximately equivalent excise taxes similarly imposed on the retail sale of other commercial tobacco products, such as cigars and chewing tobacco.

Additional restrictions placed on the sale of tobacco include a ban on cigarette vending machines in public areas accessible by juveniles, and a ban on the individual sale of single cigarettes. Revenue generated by the act was earmarked for various environmental and health care programs, and anti-tobacco advertisements. To evaluate its effect, we can gather data on cigarette sales from multiple states and across a number of years. In our case, we got data from the year 1970 to 2000 from 39 states.

## 1. Import libraries and data:

In [1]:
# Instalemos Sparse Synthetic Controls
import os
install = '"git+https://github.com/microsoft/SparseSC.git"'
os.system(f'pip install -Uqq {install}')

0

In [2]:
import pandas as pd
import numpy as np
import SparseSC
from datetime import datetime
import plotly.express as px
import plotly.graph_objects as pgo
pd.set_option("display.max_columns", None)

# Omiting WARNINGS
import warnings
warnings.filterwarnings('ignore')

In [3]:
# Read data

df = pd.read_csv("smoking_data.csv")

df.head()

,state,year,cigsale,lnincome,beer,age15to24,retprice
0,Alabama,1970.0,89.8,NaN,NaN,0.178862,39.6
1,Alabama,1971.0,95.4,NaN,NaN,0.179928,42.7
2,Alabama,1972.0,101.1,9.498476,NaN,0.180994,42.3
3,Alabama,1973.0,102.9,9.550107,NaN,0.182060,42.1
4,Alabama,1974.0,108.2,9.537163,NaN,0.183126,43.1


In [4]:
#

df.tail()

,state,year,cigsale,lnincome,beer,age15to24,retprice
1204,Wyoming,1996.0,110.3,10.016768,24.6,NaN,162.5
1205,Wyoming,1997.0,108.8,10.025613,24.6,NaN,164.1
1206,Wyoming,1998.0,102.9,NaN,NaN,NaN,168.8
1207,Wyoming,1999.0,104.8,NaN,NaN,NaN,189.6
1208,Wyoming,2000.0,90.5,NaN,NaN,NaN,267.1


In [5]:
#

df.shape

(1209, 7)

We have data per state as treatment unit and yearly (year column) per-capita sales of cigarettes in packs (cigsale column) and the cigarette retail price (retprice column). We are going to pivot this data so that each row is one treatment unit(state), and columns represent the yearly cigsale value.

In [7]:
# Pivot Table:

df = df.pivot(index = 'state', columns = 'year', values = "cigsale")

df.head()

year,1970.0,1971.0,1972.0,1973.0,1974.0,1975.0,1976.0,1977.0,1978.0,1979.0,1980.0,1981.0,1982.0,1983.0,1984.0,1985.0,1986.0,1987.0,1988.0,1989.0,1990.0,1991.0,1992.0,1993.0,1994.0,1995.0,1996.0,1997.0,1998.0,1999.0,2000.0
state,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Alabama,89.8,95.4,101.1,102.9,108.2,111.7,116.2,117.1,123.0,121.4,123.2,119.6,119.1,116.3,113.0,114.5,116.3,114.0,112.1,105.6,108.6,107.9,109.1,108.5,107.1,102.6,101.4,104.9,106.2,100.7,96.2
Arkansas,100.3,104.1,103.9,108.0,109.7,114.8,119.1,122.6,127.3,126.5,131.8,128.7,127.4,128.0,123.1,125.8,126.0,122.3,121.5,118.3,113.1,116.8,126.0,113.8,108.8,113.0,110.7,108.7,109.5,104.8,99.4
California,123.0,121.0,123.5,124.4,126.7,127.1,128.0,126.4,126.1,121.9,120.2,118.6,115.4,110.8,104.8,102.8,99.7,97.5,90.1,82.4,77.8,68.7,67.5,63.4,58.6,56.4,54.5,53.8,52.3,47.2,41.6
Colorado,124.8,125.5,134.3,137.9,132.8,131.0,134.2,132.0,129.2,131.5,131.0,133.8,130.5,125.3,119.7,112.4,109.9,102.4,94.6,88.8,87.4,90.2,88.3,88.6,89.1,85.4,83.1,81.3,81.2,79.6,73.0
Connecticut,120.0,117.6,110.8,109.3,112.4,110.2,113.4,117.3,117.5,117.4,118.0,116.4,114.7,114.1,112.5,111.0,108.5,109.0,104.8,100.6,91.5,86.7,83.5,79.1,76.6,79.3,76.0,75.9,75.5,73.4,71.4


## 2. Descriptive Analysis

Let's observe how cigarettes sales per capita is trending over time w.r.t California and other states.

In [11]:
#

plot_df = df.loc[df.index == "California"].T.reset_index(drop = False)

plot_df.head()

state,year,California
0,1970.0,123.0
1,1971.0,121.0
2,1972.0,123.5
3,1973.0,124.4
4,1974.0,126.7


In [12]:
#

plot_df["OtherStates"] = df.loc[df.index != "California"].mean(axis = 0).values

plot_df.head()

state,year,California,OtherStates
0,1970.0,123.0,120.084211
1,1971.0,121.0,123.863158
2,1972.0,123.5,129.178947
3,1973.0,124.4,131.539474
4,1974.0,126.7,134.668421


In [20]:
#
fig = px.line( data_frame = plot_df, 
               x = "year", y = ["California","OtherStates"], 
               template = "plotly_white")
# "plotly", "plotly_white", "plotly_dark", "ggplot2", "seaborn", "simple_white", "none"

fig.add_trace( pgo.Scatter( x = [1988, 1988],
                            y = [plot_df.California.min()*0.98, plot_df.OtherStates.max()*1.02],
                            line = { 'dash': 'dash', }, name = 'Proposition 99'
                          ))

fig.update_layout( title = { 'text': "Gap in per-capita cigarette sales(in packs)",
                             'y':0.95, 'x':0.5, },
                   legend = dict(y = 1, x = 0.8, orientation = 'v'),
                   legend_title = "",
                   xaxis_title = "Year", 
                   yaxis_title = "Cigarette Sales Trend",
                   font = dict(size = 15)
                 )

fig.show()

As we can see, there is a general decline in cigarette sales after the 1980s, and with the introduction of Proposition 99, the decreasing trend accelerated for the state of California. 

We cannot say for sure if this is happening with any statistical significance, it is just something we observed by examining the chart above.

To answer the question of whether Proposition 99 influenced cigarette consumption, we will use the pre-intervention period (1970-1988) to build a synthetic control group that mimics California cigarette sales trend. Then, we will see how this synthetic control behaves after the intervention.

## Fitting Synthetic Control using SparseSC package

In [21]:
# Algunas Funciones previas:

df.iloc[ : , df.columns <= 1988]#.values

year,1970.0,1971.0,1972.0,1973.0,1974.0,1975.0,1976.0,1977.0,1978.0,1979.0,1980.0,1981.0,1982.0,1983.0,1984.0,1985.0,1986.0,1987.0,1988.0
state,,,,,,,,,,,,,,,,,,,
Alabama,89.8,95.4,101.1,102.9,108.2,111.7,116.2,117.1,123.0,121.4,123.2,119.6,119.1,116.3,113.0,114.5,116.3,114.0,112.1
Arkansas,100.3,104.1,103.9,108.0,109.7,114.8,119.1,122.6,127.3,126.5,131.8,128.7,127.4,128.0,123.1,125.8,126.0,122.3,121.5
California,123.0,121.0,123.5,124.4,126.7,127.1,128.0,126.4,126.1,121.9,120.2,118.6,115.4,110.8,104.8,102.8,99.7,97.5,90.1
Colorado,124.8,125.5,134.3,137.9,132.8,131.0,134.2,132.0,129.2,131.5,131.0,133.8,130.5,125.3,119.7,112.4,109.9,102.4,94.6
Connecticut,120.0,117.6,110.8,109.3,112.4,110.2,113.4,117.3,117.5,117.4,118.0,116.4,114.7,114.1,112.5,111.0,108.5,109.0,104.8
Delaware,155.0,161.1,156.3,154.7,151.3,147.6,153.0,153.3,155.5,150.2,150.5,152.6,154.1,149.6,144.0,144.5,142.4,141.0,137.1
Georgia,109.9,115.7,117.0,119.8,123.7,122.9,125.9,127.9,130.6,131.0,134.0,131.7,131.2,128.6,126.3,128.8,129.0,129.3,124.1
Idaho,102.4,108.5,126.1,121.8,125.6,123.3,125.1,125.0,122.8,117.5,115.2,114.1,111.5,111.3,103.6,100.7,96.7,95.0,84.5
Illinois,124.8,125.6,126.6,124.4,131.9,131.8,134.4,134.0,136.7,135.3,135.2,133.0,130.7,127.9,124.0,121.6,118.2,109.5,107.6


In [22]:
# Algunas funciones previas

df.iloc[ : ,df.columns > 1988]#.values

year,1989.0,1990.0,1991.0,1992.0,1993.0,1994.0,1995.0,1996.0,1997.0,1998.0,1999.0,2000.0
state,,,,,,,,,,,,
Alabama,105.6,108.6,107.9,109.1,108.5,107.1,102.6,101.4,104.9,106.2,100.7,96.2
Arkansas,118.3,113.1,116.8,126.0,113.8,108.8,113.0,110.7,108.7,109.5,104.8,99.4
California,82.4,77.8,68.7,67.5,63.4,58.6,56.4,54.5,53.8,52.3,47.2,41.6
Colorado,88.8,87.4,90.2,88.3,88.6,89.1,85.4,83.1,81.3,81.2,79.6,73.0
Connecticut,100.6,91.5,86.7,83.5,79.1,76.6,79.3,76.0,75.9,75.5,73.4,71.4
Delaware,131.7,127.2,118.8,120.0,123.8,126.1,127.2,128.3,124.1,132.8,139.5,140.7
Georgia,117.1,113.8,109.6,109.2,109.2,107.8,100.3,102.7,100.6,100.5,97.1,88.4
Idaho,78.4,90.1,85.4,85.1,86.7,93.0,78.2,73.6,75.0,78.9,75.1,66.9
Illinois,104.6,94.1,96.1,94.8,94.6,85.7,84.3,81.8,79.6,80.3,72.2,70.0


In [23]:
# Algunas funciones previas

df.index.values

array(['Alabama', 'Arkansas', 'California', 'Colorado', 'Connecticut',
       'Delaware', 'Georgia', 'Idaho', 'Illinois', 'Indiana', 'Iowa',
       'Kansas', 'Kentucky', 'Louisiana', 'Maine', 'Minnesota',
       'Mississippi', 'Missouri', 'Montana', 'Nebraska', 'Nevada',
       'New Hampshire', 'New Mexico', 'North Carolina', 'North Dakota',
       'Ohio', 'Oklahoma', 'Pennsylvania', 'Rhode Island',
       'South Carolina', 'South Dakota', 'Tennessee', 'Texas', 'Utah',
       'Vermont', 'Virginia', 'West Virginia', 'Wisconsin', 'Wyoming'],
      dtype=object)

On a high level SparseSC package provide two functions for fitting Synthetic Controls (SCs) i.e., fit() method and fit_fast() method. 

On a high level:

* fit() - This method tries to compute the weight jointly and results in SCs which are 'optimal'. This is the most common method used in most of the code/libraries I have found but it is computationally expensive and takes a long time to run. Hence, does not scale for larger datasets.

* fit_fast()- This method tries to compute the weight separately by performing some non-matching analysis. This solution is much faster and often the only feasible method with larger datasets. The authors of this package recommend the fit_fast method to start with and only move towards the fit method if needed.

The SparseSC.fit_fast() method required at least three arguments:

- features - This is the NumPy matrix of I/p variables where each row represents a treatment/control unit (states in our case), each column is the period from pre-treatment (1970-1988), and the value in the matrix is the metric of interest (in this case it is the cigsale value)

- targets - This is the NumPy matrix of I/p variables where each row represents a treatment/control unit (states in our case), each column is the period from post-treatment (1999-2000), and the value in the matrix is the metric of interest (in this case it is the cigsale value)

- treatment_units - This is the list of integers containing the row index value of treated units

See: https://github.com/microsoft/SparseSC, for details.

**Note : That treatment units can be a list of multiple treatment indexes. Think of cases where the same treatment is applied to multiple groups, for example, what if proposition 99 was rolled in both California and Minnesota State, in this case, treatment_units will get [2, 15], which are the respective index of these states.**

In [24]:
## creating required features
features      = df.iloc[ : , df.columns <= 1988].values
targets       = df.iloc[ : , df.columns > 1988].values
treated_units = [idx for idx, val in enumerate(df.index.values) if val == 'California'] # [2]

## Fit fast model for fitting Synthetic controls
sc_model = SparseSC.fit_fast( features      = features,
                              targets       = targets,
                              treated_units = treated_units )

Now that we have fitted the model, let's get the Synthetic Control output by using predict() function.

In [25]:
#

result = df.loc[ df.index == 'California' ].T.reset_index( drop = False )

result.columns = ["year", "Observed"] 

result.head()

,year,Observed
0,1970.0,123.0
1,1971.0,121.0
2,1972.0,123.5
3,1973.0,124.4
4,1974.0,126.7


In [26]:
# Weights:

W = sc_model.get_weights( include_trivial_donors = True )

print(W)

[[ 0.          0.09735335 -0.02251586 ...  0.03486265  0.03882214
  -0.03603962]
 [ 0.10244426  0.         -0.01961628 ...  0.06752741  0.05354457
  -0.01373064]
 [-0.02082824 -0.0175402   0.07073677 ...  0.02058647  0.02321142
   0.02110959]
 ...
 [ 0.03094019  0.0602389  -0.0207691  ...  0.          0.03543806
   0.05483416]
 [ 0.03408686  0.04438965  0.00753188 ...  0.03478619  0.
  -0.00556783]
 [-0.04445077 -0.01991944  0.08711779 ...  0.06707758 -0.00546119
   0.        ]]


In [27]:
#

W.shape

(39, 38)

In [28]:
# Predict:

result['Synthetic'] = sc_model.predict( df.values )[ treated_units, : ][0]

result.head(5)

,year,Observed,Synthetic
0,1970.0,123.0,122.394195
1,1971.0,121.0,125.114849
2,1972.0,123.5,129.704372
3,1973.0,124.4,126.753988
4,1974.0,126.7,126.276394


Now that we have our synthetic control, we can plot it with the outcome variable of the State of California.

In [29]:
# Plot:
fig = px.line( data_frame = result, 
               x = "year", y = ["Observed" , "Synthetic"], 
               template = "plotly_white", )

fig.add_trace( pgo.Scatter( x = [1988, 1988],
                            y = [result.Observed.min()*0.98, result.Observed.max()*1.02], 
                            line = { 'dash': 'dash', }, name = 'Proposition 99' 
                          ))

fig.add_trace( pgo.Scatter( x = result['year'],
                            y = plot_df['OtherStates'], name = 'Others (mean)' 
                          ))

fig.update_layout( title = { 'text':"Synthetic Control Assessment", 
                             'y':0.95, 'x':0.5, }, 
                   legend = dict( y = 1, x = 0.8, orientation = 'v'),
                   legend_title = "",
                   xaxis_title = "Year", yaxis_title = "Per-capita cigarette sales (in packs)",
                   font = dict( size = 15 ) )

fig.show()

In the pre-intervention period, the synthetic control does not reproduce the treatment exactly but follows the curve closely. This is a good sign, as it indicates that we are not overfitting. Also, note that we do see divergence after the intervention (introduction of Proposition 99) after 1988.

With the synthetic control groups in hand, we can estimate the treatment effect as the gap between the treated and the synthetic control outcomes.

In [30]:
# Fig - Gap in Per-capita cigarette sales in California w.r.t Synthetic Control

result['California Effect'] = result.Observed - result.Synthetic

fig = px.line( data_frame = result, 
               x = "year", y = "California Effect", 
               template = "plotly_white",)

fig.add_hline(0)

fig.add_trace( pgo.Scatter( 
               x = [ 1988, 1988 ], 
               y = [result["California Effect"].min()*0.98, result["California Effect"].max()*1.02], 
               line = { 'dash': 'dash', }, name='Proposition 99' ))

fig.update_layout( title = { 'text':"Difference across time", 
                             'y':0.95, 'x':0.5, },
                             legend =  dict(y = 1, x = 0.8, orientation = 'v'),
                             legend_title = "",
                             xaxis_title = "Year", 
                             yaxis_title = "Gap in Per-capita cigarette sales (in packs)",
                             font = dict( size = 15 ))

fig.show()

In [31]:
print( f"Effect of Proposition 99 w.r.t Synthetic Control => \
       {np.round(result.loc[result.year == 2000, 'California Effect'].values[0],1)} packs")


Effect of Proposition 99 w.r.t Synthetic Control =>        -28.8 packs
